# Proyecto Webscraping
Eres parte de un equipo de análisis de datos encargado de investigar y comparar los precios de compra de casas en distintas ciudades de Uruguay. Para ello, deberás scrapear una página web inmobiliaria que provea datos de propiedades en venta, como los precios, la ubicación y otros detalles relevantes.

Tu objetivo es realizar un scraping de los datos de las propiedades en venta en distintas ciudades de Uruguay y almacenarlos en un archivo JSON para su posterior análisis.

#### Importar las librerías necesarias

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

## Agregar el URL de la inmobiliaria Infocasas

In [ ]:
url = 'https://www.infocasas.com.uy/alquiler/apartamentos'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

soup

### Añadir a la URL más páginas, para así obtener más propiedades

In [ ]:
base_url = 'https://www.infocasas.com.uy/alquiler/apartamentos'
for page in range (1, 20):
    # https://www.infocasas.com.uy/alquiler/apartamentos/pagina2
    url = f'{base_url}/pagina{page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

soup

### Encontrar los Links de cada propiedad

In [ ]:
links = soup.find_all('a', class_='lc-data')
for link in links:
    print(link.get('href'))

#### Unimos el 'href' con la URL para que nos de el link completo

In [ ]:
links_soup = soup.find_all('a', class_='lc-data')
all_links = []

for l in links_soup:
    link = base_url + l['href']
    all_links.append(link)

all_links

### Encontrar la Ciudad en la cuál se encuentra la propiedad

In [ ]:
locations = soup.find_all('strong', class_="lc-location body body-1 body-bold medium")

foo = [location.text.strip() for location in locations]
cities = [city.split(',')[1] for city in foo]
cities

### Encontrar la cantidad de Habitaciones por propiedad

In [ ]:
rooms = soup.find_all('strong')
dorms = []
for r in rooms:
    texto = r.text.strip()
    if "Dorm" in texto or "Dorms" in texto:
        dorms.append(texto.strip().split()[0])

dorms

### Encontrar el Precio de cada propiedad

In [ ]:
price_soup = soup.find_all('p', class_='main-price')
prices = []
for price in price_soup:
    prices.append(price.text.strip().split()[1])

prices

#### La consigna pide los precios en dolares, por lo que lo transformamos...

In [ ]:
price_soup = soup.find_all('p', class_='main-price')
tipo_cambio = 39.88
prices = []
for price in price_soup:
    texto = price.text.strip()
    partes = texto.split()
    simbolo = partes[0]
    valor = partes[1]
    if simbolo == '$':
        valor_numerico = float(valor.replace('.', '').replace(',', '.'))
        valor_final = valor_numerico * tipo_cambio
        valor = valor_final
    prices.append(valor)

prices

### Encontra el tamaño en m²

In [ ]:
tamanos_soup = soup.find_all('strong')
tamanos = []
for tamano in tamanos_soup:
    texto = tamano.text.strip()
    if "m²" in texto:
        tamanos.append(texto.strip().split()[0])

tamanos

### Crear un diccionario que tenga la información para cada propiedad

In [ ]:
propiedades = []
# Utilice la función zip para unir las listas en una sola secuencia y así poder agrupar los elementos que están en la misma posición
# En cada bucle, se obtiene un paquete de información para cada propiedad
for ciudad, precio, dormitorios, tamano, link in zip(cities, prices, dorms, tamanos, all_links):
    propiedad = {
        "precio": precio,
        "habitaciones": dormitorios,
        "tamano": tamano,
        "link": link
    }
    propiedades.append(propiedad)

propiedades

### Añadir esa información a un diccionario que su clave es la ciudad
Con este nuevo diccionario, se pueden agrupar las propiedades por ciudad

In [ ]:
# Crear un diccionario vacío para luego guardar las propiedades agrupadas por su respetiva ciudad
ciudades_dict= {}
for ciudad, precio, dormitorios, tamano, link in zip(cities, prices, dorms, tamanos, all_links):
    propiedad = {
        "precio": precio,
        "habitaciones": dormitorios,
        "tamano": tamano,
        "link": link
    }

# Si la ciudad aún no está en el diccionario, se crea una clave con esa ciudad y se le asigna una lista vacía
# Luego, la propiedad que corresponde se le asigna a la lista de esa ciudad
    if ciudad not in ciudades_dict:
         ciudades_dict[ciudad] = []
    ciudades_dict[ciudad].append(propiedad)

# Recorre todas las ciudades que están en el diccionario. Y se queda con las primeras 10 propiedades.
for ciudad in ciudades_dict:
    ciudades_dict[ciudad] = ciudades_dict[ciudad][:10]

ciudades_dict

### Transformar el diccionario en el formato que pide la consigna...
Crear un nuevo diccionario (resultado), que contiene la clave 'ciudades', donde se encuentra la lista de todas las ciudades y sus respectivas propiedades

In [ ]:
resultado = {
    'ciudades': [
        {
            'nombre': ciudad,
            'propiedades': propiedades
        }
        for ciudad, propiedades in ciudades_dict.items()
    ]
}

In [ ]:
with open('ciudades_dict.json', 'w') as outfile:
    json.dump(resultado, outfile, indent=4)

#### Fórmula para saber la cantidad de ciudades
Ya que por ejemplo para Canelones o Maldonado, son menos de 10 propiedades

In [ ]:
from collections import Counter

conteo_ciudades = Counter(cities)
print(conteo_ciudades)